In [3]:

import torch
import copy
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.ao.quantization.quantize_fx as quantize_fx
from src.vgg import *
from src.util import *
import os

def get_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    return os.path.getsize("temp.p")/1e6

model_path = "./models"

vgg11 = vgg('vgg11'); vgg11.load_state_dict(torch.load(f'{model_path}/vgg11.pth')); vgg11.eval()
vgg13 = vgg('vgg13'); vgg13.load_state_dict(torch.load(f'{model_path}/vgg13.pth')); vgg13.eval()
vgg16 = vgg('vgg16'); vgg16.load_state_dict(torch.load(f'{model_path}/vgg16.pth')); vgg16.eval()
vgg19 = vgg('vgg19'); vgg19.load_state_dict(torch.load(f'{model_path}/vgg19.pth')); vgg19.eval()

models = {
    'vgg11': vgg11,
    'vgg13': vgg13,
    'vgg16': vgg16,
    'vgg19': vgg19,
}

In [4]:
# measure model size
scale = 1024**2
for model_name, model in models.items():
    model_size = get_size_of_model(model)
    print(f'{model_name} model size:\t {model_size} MB')

vgg11 model size:	 36.961198 MB
vgg13 model size:	 37.705754 MB
vgg16 model size:	 58.970204 MB
vgg19 model size:	 80.234732 MB


In [5]:
# measure inference latency
for model_name, model in models.items():
    elapsed_time = measure_inference_latency(model, 'cpu')
    print(f'{model_name} elapsed time:\t {elapsed_time:4f}')

vgg11 elapsed time:	 0.002370
vgg13 elapsed time:	 0.002467
vgg16 elapsed time:	 0.003432
vgg19 elapsed time:	 0.004339


In [6]:
# measure accuracy

data_path = "/workspace/shared/data"
test_dataset = datasets.CIFAR10(root=data_path, train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for model_name, model in models.items():
    accuracy = measure_accuracy(model, test_loader, 'cpu')
    print(f'{model_name} accuracy:\t {accuracy:.4f}')

Files already downloaded and verified
vgg11 accuracy:	 0.8568
vgg13 accuracy:	 0.8719
vgg16 accuracy:	 0.8728
vgg19 accuracy:	 0.8762
